In [5]:
import http.client as httplib
from pymodbus.client import ModbusTcpClient
import serial
from pynq import PL, Overlay
from pynq import allocate
import new_det_data_acquisition_common as common
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os
%matplotlib inline
import socket
from pymodbus.constants import Endian
from pymodbus.payload import BinaryPayloadDecoder

/home/susmit/.local/lib/python3.10/site-packages/pydantic/_internal/_config.py:321: UserWarning: Valid config keys have changed in V2:
* 'underscore_attrs_are_private' has been removed
  warnings.warn(message, UserWarning)
/home/susmit/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [10]:
ip = '192.168.0.113'
port = 502

register_address_map = {"TEMP_PV": 0,
                        "RH_PV": 2,
                        "RH_SV_Read_Only": 3,
                        "Program_Status": 4,
                        "TEMP_SV": 7,
                        "RH_SV_Read_Write": 16
                        } 
coil_address_map = {"Profile_Run": 0,
                    "Lamp_Command": 1,
                    "Manual_Run" : 2,
                    "Batch_Start_Flag": 3
                   }

In [11]:
client = ModbusTcpClient(host=ip, port=port)
connection = client.connect()

Connection to (192.168.0.113, 502) failed: timed out


In [13]:
while True:
    try:
        for key, value in coil_address_map.items():
            rd = client.read_coils(value)
            print()
            print(rd)
            print(f"{key: <10}: {rd.bits}")
        break
    except Exception as e:
        break

Connection to (192.168.0.113, 502) failed: timed out


In [ ]:
while True:
    try:
        rq = client.write_register(16, 50)       # RH_SV_Read_Write = 70 %
        rd = client.read_holding_registers(value)
        print(rd)
        break
    except:
        break

In [ ]:
print("\nReading holding registers:")
while True:
    try:
        for key, value in register_address_map.items():
            rd = client.read_holding_registers(value)
            print()
            print(rd)
            print(f"{key: <10}: {rd.registers}")
        break
    except:
        break

In [1]:
print("\nWriting to set value holding registers:\n")

tsv=int(input("Write TEMP_SV in C"))
rhsv=int(input("Write RH_SV"))

while True:
    try:
        key = "TEMP_SV"
        value = register_address_map[key]
        rq = client.write_register(value, tsv*10)       # TEMP_SV = 20.0 deg C
        rd = client.read_holding_registers(value)
        print(rd)
        print(f"{key: <10}: {rd.registers}")
        rhsv_binary = format(rhsv, '032b')

        rhsv_high_binary = rhsv_binary[:16]
        rhsv_low_binary = rhsv_binary[16:]
        rhsv_high = int(rhsv_high_binary, 2)
        rhsv_low = int(rhsv_low_binary, 2)
        key = "RH_SV_Read_Write"
        value_l=register_address_map[key]
        value_h=value_l+1
        
        rq_l=client.write_register(value_l, rhsv_low)
        rd_l=client.read_holding_registers(value_l)
        rq_h=client.write_register(value_h, rhsv_high)
        rd_h=client.write_register(value_h)
        
        break
    except:
        pass


Writing to set value holding registers:



Write TEMP_SV in C 10
Write RH_SV 50


KeyboardInterrupt: 

In [2]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m' 
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [6]:
ov = Overlay("./overlays/test_2det_commanding.bit")
# czt = ov.AXI_CZT_AXIS_0.AXI_CZT_Controller                # Bottom slot on PYNQ board
czt = ov.AXI_CZT_AXIS_1.AXI_CZT_Controller      

/home/susmit/.local/lib/python3.10/site-packages/pynq/pl_server/device.py:56: UserWarning: No devices found, is the XRT environment sourced?
  warnings.warn(


RuntimeError: No Devices Found

In [7]:
det=czt.read_serial

NameError: name 'czt' is not defined